![Trolo](https://private-user-images.githubusercontent.com/15766192/391220869-08091cad-703d-4d7d-84f6-5e6c310bf6c2.png?jwt=eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.*eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmF3LmdpdGh1YnVzZXJjb250ZW50LmNvbSIsImtleSI6ImtleTUiLCJleHAiOjE3MzMzMTMzOTksIm5iZiI6MTczMzMxMzA5OSwicGF0aCI6Ii8xNTc2NjE5Mi8zOTEyMjA4NjktMDgwOTFjYWQtNzAzZC00ZDdkLTg0ZjYtNWU2YzMxMGJmNmMyLnBuZz9YLUFtei1BbGdvcml0aG09QVdTNC1ITUFDLVNIQTI1NiZYLUFtei1DcmVkZW50aWFsPUFLSUFWQ09EWUxTQTUzUFFLNFpBJTJGMjAyNDEyMDQlMkZ1cy1lYXN0LTElMkZzMyUyRmF3czRfcmVxdWVzdCZYLUFtei1EYXRlPTIwMjQxMjA0VDExNTEzOVomWC1BbXotRXhwaXJlcz0zMDAmWC1BbXotU2lnbmF0dXJlPTQ4NDAwOTc2ZGNlMDQ1YmMwMWExMzBjYmE5NjJmMGYxMTE1NzY1ZDI0NGZkMTI5N2Q0YzRlMGJiMDc5ZDI2ZjgmWC1BbXotU2lnbmVkSGVhZGVycz1ob3N0In0*.H8dvVZt5CImX2vEg2WP69FX3e3uAXIsuvo4TRjyQgE4)

# Serialise Detections to a CSV
---
[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ayushexel/trolo/blob/master/recipes/detect-small-objects.ipynb)

## Installation
---
#### Installing the Trolo package directly from the GitHub repo


In [ ]:
!pip install git+https://github.com/AyushExel/trolo.git

## Imports
---

In [2]:
import csv
from typing import List
from collections import defaultdict

import numpy as np
import pandas as pd

from PIL import Image

import supervision as sv
from supervision.assets import download_assets, VideoAssets

## Download Image
---
#### As an example, we will use a picture from the internet.


In [ ]:
import os
HOME = os.getcwd()

download_assets(VideoAssets.PEOPLE_WALKING)

FILE_NAME = f"/{HOME}/detections.csv"
SOURCE_VIDEO_PATH = f"/{HOME}/people-walking.mp4"

sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

## Initialize Detection and Supervision Objects
---

In [4]:
from trolo.inference import DetectionPredictor

predictor = DetectionPredictor(model="dfine-s")

🛈 Downloading model from https://github.com/ayushexel/trolo/releases/download/0.1.1/dfine-s.pth
❌ Failed to download dfine-s.pth: 404 Client Error: Not Found for url: https://github.com/ayushexel/trolo/releases/download/0.1.1/dfine-s.pth
🛈 Downloading model from https://github.com/ayushexel/trolo/releases/download/0.1.1/dfine_s.pth
🛈 Successfully downloaded model to dfine_s.pth
🛈 Loading config from checkpoint


In [5]:
byte_track = sv.ByteTrack(minimum_consecutive_frames=3)
byte_track.reset()

csv_sink = sv.CSVSink(FILE_NAME)

## Process Video and Save Detections to CSV
---

In [6]:
def callback(frame: np.array, index:int):

    prediction = predictor.predict(Image.fromarray(frame))

    detections = sv.Detections(
        xyxy=prediction[0]['boxes'].numpy(),
        confidence=prediction[0]['scores'].numpy(),
        class_id=prediction[0]['labels'].numpy()
    )
    detections = byte_track.update_with_detections(detections)

    detections = detections[detections.class_id == 0]
    detections.data["class_name"] = np.array(["person" for _ in range(len(detections))])

    csv_sink.append(detections, custom_data={'frame_number': index})

In [7]:
TARGET_VIDEO_PATH = f'/{HOME}/target.mp4'

csv_sink.open()
sv.process_video(
    SOURCE_VIDEO_PATH,
    TARGET_VIDEO_PATH,
    callback=callback
)
csv_sink.close()

# Visalize the detections from CSV
---

In [ ]:
df = pd.read_csv(FILE_NAME)
df